## 一、Mongodb基础使用教程
1. 安装Mongodb数据库
2. 安装Robomongo可视化数据库
3. 安装pymongo交互工具
4. 安装datautils方法封装
5. 导入Excel数据
6. 数据库数据读取

1. 安装Mongodb: https://www.mongodb.com/download-center/community
2. 安装Robomongo: http://mongodb-tools.com/tool/robomongo/
3. 安装pymongo：pip install pymongo

## 数据库基本知识
    1. Client: host, port
    2. DataBase
    3. Collections

## 5. 导入Excel
### 相对路径

./当前路径

../上一个目录

../../上上个目录

In [1]:
import pymongo
import pandas as pd
from datetime import datetime

In [2]:
dataCsv = pd.read_csv('../IF88.csv')

In [4]:
dataCsv.tail(1)

,Unnamed: 0,_id,close,date,datetime,exchange,high,low,open,openInterest,symbol,time,trading_date,volume,vtSymbol
365789,365789,5e16c1cb8fb6b8371667074e,4090.8,20191230,2019-12-30 14:59:00,CTP,4091.6,4089.6,4090.4,82058,IF88,14:59:00,2019-12-30,968,IF88:CTP


In [5]:
dataCsv['datetime'].iloc[0]

'2014-01-02 09:15:00'

In [6]:
dataCsv['datetime'] = list(map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"), dataCsv['datetime']))

In [8]:
symbol = 'IF88:CTP'
client = pymongo.MongoClient('localhost', 33333)

collection = client['VnTrader_1Min_Db_Case'][symbol]

############################
collection.create_index([('datetime', pymongo.ASCENDING)], unique=True)

'datetime_1'

In [9]:
for index, row in dataCsv.iterrows():
    bar = {}
    bar['open'] = row.open
    bar['close'] = row.close
    bar['high'] = row.high
    bar['low'] = row.low
    bar['volume'] = row.volume
    bar['symbol'] = symbol
    bar['vtSymbol'] = symbol
    bar['datetime'] = row.datetime
    bar['date'] = bar['datetime'].date().strftime('%Y%m%d')
    bar['time'] = bar['datetime'].time().strftime('%H:%M:%S')
    ######################################
    flt = {'datetime': bar['datetime']}
    ######################################
    collection.update_one(flt, {'$set':bar}, upsert=True)

In [10]:
collection.update_one?

## 数据库常用命令
sort/find

## 6. 数据库数据读取

In [12]:
from datetime import datetime

def readAll(collection, start, end):
    return pd.DataFrame(
        list(collection.find({"datetime": {'$gt': start, '$lt': end}}))
    )

In [21]:
IF88 = readAll(client['VnTrader_1Min_Db_Case']['IF88:CTP'], datetime(2019,11,1), datetime(2019,12,31))

In [25]:
IF88 = IF88.set_index('datetime')

7. 数据合并处理
    1. resample
    2. agg

In [26]:
def resampleDf(data, freq='H'):
    return data.resample(freq).agg({'open':'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume':'sum'})

In [27]:
dataResample = resampleDf(IF88, '5Min')

In [28]:
dataResample.tail()

,open,high,low,close,volume
datetime,,,,,
2019-12-30 14:35:00,4094.0,4098.2,4092.0,4093.6,1112
2019-12-30 14:40:00,4094.4,4094.4,4088.4,4091.6,1244
2019-12-30 14:45:00,4091.6,4098.6,4090.6,4096.8,1120
2019-12-30 14:50:00,4096.2,4096.2,4088.2,4092.0,1613
2019-12-30 14:55:00,4092.0,4092.2,4088.6,4090.8,2885


# 习题
1. 写一个循环来导入三个数字货币的数据
2. 将最近一个月BTC数据从数据库读出并合成5分钟信号